In [1]:
#####################################################################
# Mandatory : Run this cell and restart the notebook kernel right after
#####################################################################
!pip install torch==1.11 transformers

In [2]:
%cd /content

[Errno 2] No such file or directory: '/content'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [1]:
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
%cd deepqa/notebooks/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8937k  100 8937k    0     0  3555k      0  0:00:02  0:00:02 --:--:-- 3555k


Archive:  squad1.1.zip
  inflating: data/dev-v1.1.json      
  inflating: data/train-v1.1.json    


In [2]:
%%bash
# Dowload the SQuAD1.1 data
curl -O https://data.deepai.org/squad1.1.zip
unzip -o squad1.1.zip -d data

2022-07-30 00:36:05.538503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-30 00:36:05.538577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import sys
import os
import pathlib

import json

import pandas as pd
import numpy as np
import torch

from tqdm import tqdm

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

deepqa_lib_path = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.insert(0, deepqa_lib_path)

from deepqa import preprocessing, tokenizer, model, wrapper, loss, metrics, utils

2022-08-01 00:58:03.418964: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 00:58:03.419034: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Read train and dev data
with open("data/train-v1.1.json", "rb") as j:
    train_data = json.load(j)["data"]
with open("data/dev-v1.1.json", "rb") as j:
    dev_data = json.load(j)["data"]

In [4]:
_, contexts, queries, answers = preprocessing.tabularize_squad11_data(train_data)
hf_tokenizer = tokenizer.HFAutoTokenizer("bert-base-uncased").word_tokenizer

tokenizer_res = hf_tokenizer(
    contexts, 
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
    return_offsets_mapping=True
    )

tokenizer_res

{'input_ids': tensor([[  101,  6549,  2135,  ...,     0,     0,     0],
        [  101,  6549,  2135,  ...,     0,     0,     0],
        [  101,  6549,  2135,  ...,     0,     0,     0],
        ...,
        [  101, 28045,  4956,  ...,     0,     0,     0],
        [  101, 28045,  4956,  ...,     0,     0,     0],
        [  101, 28045,  4956,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'offset_mapping': tensor([[[ 0,  0],
         [ 0, 13],
         [13, 15],
         ...,
         [ 0,  0],
         [ 0,  0],
         [ 0,  0]],

 

In [5]:
corpus = contexts
offset_mappings = tokenizer_res["offset_mapping"]
char_token_id_mapping = tokenizer.TokenCharTokenizer("bert-base-uncased").char_token_id_mapping
l = len(char_token_id_mapping) + 1
off_map = offset_mappings.detach().numpy()

In [8]:
%%time

acc = [[list(map(char_token_id_mapping.get, 
                 text[slice(a := off_map[i,j,0], b := off_map[i,j,1])])) + 
        (32 + a - b) * [l]
        for j in range(offset_mappings.shape[1])]
       for i, text in enumerate(corpus[0:10000])]

acc = np.asarray(acc, dtype=np.int16)

CPU times: user 39.3 s, sys: 643 ms, total: 39.9 s
Wall time: 39.9 s


In [20]:
acc.shape

torch.Size([8, 512, 32])

In [11]:
acc = torch.from_numpy(acc)

In [ ]:
%%timeit

acc = [[list(map(char_token_id_mapping.get, 
                 text[slice(a := off_map[i,j,0], b := off_map[i,j,1])])) + 
        (32 + a - b) * [l]
        for j in range(offset_mappings.shape[1])]
       for i, text in enumerate(corpus[:1000])]

In [43]:
print(torch.nn.utils.rnn.pad_sequence([torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(list(map(char_token_id_mapping.get, 
                                    text[offset_mappings[i,j,0]:offset_mappings[i,j,1]])))
                for j in range(offset_mappings.shape[1])],
            False,
            len(char_token_id_mapping) + 2
        )
       for i, text in enumerate(corpus[0:100])],
       True,
       len(char_token_id_mapping) + 2).shape)

KeyboardInterrupt: 

In [172]:
acc = [torch.tensor([c if c is not None else len(char_token_id_mapping.keys()) + 1
        for c in list(map(char_token_id_mapping.get, list(text)))])
       for text in corpus]

In [96]:
acc2 = torch.nn.utils.rnn.pad_sequence(
    [torch.nn.utils.rnn.pad_sequence(
        list(map(acc[i].__getitem__, 
             [slice(offset_mappings[i,j,0], offset_mappings[i,j,1])
              for j in range(offset_mappings.shape[1])])),
        False,
        len(char_token_id_mapping) + 2
     )
     for i in range(len(acc))],
    True,
    len(char_token_id_mapping) + 2
)
acc2 = acc2.transpose(2, 1)
acc2.shape

KeyboardInterrupt: 

In [202]:
# Prototype CharTokenizer
s = set()
for text in contexts + queries:
    s = s.union(set(text))
s

{'ಚ',
 '籲',
 '族',
 '子',
 'キ',
 'ḱ',
 'Í',
 '松',
 'Ⲭ',
 '不',
 'Đ',
 'å',
 'ม',
 'ս',
 'К',
 'Ž',
 '导',
 'カ',
 'α',
 '\ufeff',
 '7',
 '他',
 '♯',
 'া',
 '華',
 '有',
 'Ć',
 'リ',
 'ᵻ',
 '부',
 'つ',
 'ت',
 '千',
 'ů',
 'ا',
 '豔',
 '漳',
 'ề',
 '弘',
 '仔',
 'ā',
 '韓',
 'い',
 '₤',
 '瓦',
 'ِ',
 '→',
 'ə',
 '林',
 'Ꮿ',
 '帝',
 '布',
 'i',
 '越',
 'ä',
 '≈',
 '院',
 '4',
 'য',
 'Ħ',
 '寧',
 'ײ',
 '古',
 'ﬂ',
 '主',
 'Е',
 'ồ',
 'န',
 'ח',
 'ἵ',
 'ย',
 '船',
 '@',
 '물',
 '(',
 "'",
 'ˌ',
 'ʀ',
 'ť',
 '湖',
 '用',
 'ʘ',
 'ง',
 '|',
 '신',
 'V',
 'Ẓ',
 'v',
 'さ',
 'Ἰ',
 'ḷ',
 'ֹ',
 'ʲ',
 '文',
 'ü',
 '𒆳',
 'ɸ',
 '∈',
 '語',
 '魄',
 'ƿ',
 '˚',
 'X',
 '地',
 'ล',
 'æ',
 '澄',
 'Θ',
 'ब',
 'ಿ',
 's',
 'ñ',
 'ç',
 '良',
 'բ',
 '體',
 '비',
 '¡',
 '媠',
 'υ',
 'ܐ',
 'Ј',
 'ξ',
 'ָ',
 '√',
 'Ṣ',
 '平',
 'デ',
 'ỉ',
 '肉',
 'น',
 '〜',
 'ว',
 '郎',
 'ѭ',
 ']',
 'ⲏ',
 'U',
 '˩',
 '時',
 '京',
 '͡',
 'и',
 '常',
 'ř',
 '藏',
 'Ꭶ',
 '♆',
 '覆',
 '∗',
 'イ',
 'g',
 '堂',
 '重',
 '膺',
 'Ō',
 '0',
 '风',
 'ǎ',
 '蝶',
 'ǔ',
 '್',
 '印',
 'ἐ',
 '普',
 '浦',

In [203]:
import pickle

with open("../tokenizers_configs/bert-base-uncased.ctk", "wb+") as f:
    pickle.dump(s, f)